# FACT_CONSUMPTION_WEEKLY - Weekly Aggregated Consumption Table

Weekly aggregated consumption and revenue metrics for week-over-week analysis in the Genie Space.

**Note:** This table is populated by aggregating from `fact_consumption_daily`.

## 1. Create Table Schema

In [ ]:
%sql
-- Drop table if exists (uncomment if needed)
-- DROP TABLE IF EXISTS ankit_yadav.demo.fact_consumption_weekly;

CREATE TABLE IF NOT EXISTS ankit_yadav.demo.fact_consumption_weekly (
  year_week STRING COMMENT 'Year and week number (e.g., 2025-W05)',
  week_start_date DATE COMMENT 'First day of the week (Monday)',
  week_end_date DATE COMMENT 'Last day of the week (Sunday)',
  account_id STRING NOT NULL COMMENT 'Foreign key to dim_accounts',
  workspace_id STRING COMMENT 'Databricks workspace identifier',
  platform STRING COMMENT 'Cloud platform (AWS, Azure, GCP)',
  sku STRING COMMENT 'Product SKU',
  account_name STRING COMMENT 'Customer account name',
  account_executive STRING COMMENT 'Account Executive name',
  account_executive_manager STRING COMMENT 'AE Manager name',
  workspace_name STRING COMMENT 'Workspace display name',
  total_revenue_dollars FLOAT COMMENT 'Total revenue for the week in dollars',
  total_dbus_consumed FLOAT COMMENT 'Total DBUs consumed for the week',
  avg_list_price_per_dbu FLOAT COMMENT 'Average list price per DBU',
  consumption_days_count INT COMMENT 'Number of days with consumption in the week',
  last_updated_timestamp TIMESTAMP COMMENT 'Last time this record was updated'
)
COMMENT 'Weekly aggregated consumption and revenue metrics for week-over-week analysis';

## 2. Populate from Daily Data

Aggregate daily consumption data into weekly summaries.

In [ ]:
%sql
-- Clear existing data for demo accounts
DELETE FROM ankit_yadav.demo.fact_consumption_weekly WHERE account_id LIKE 'ACCT%';

-- Aggregate from daily to weekly
INSERT INTO ankit_yadav.demo.fact_consumption_weekly
SELECT
  CONCAT(YEAR(DATE_TRUNC('WEEK', consumption_date)), '-W', LPAD(WEEKOFYEAR(DATE_TRUNC('WEEK', consumption_date)), 2, '0')) as year_week,
  DATE_TRUNC('WEEK', consumption_date) as week_start_date,
  DATE_ADD(DATE_TRUNC('WEEK', consumption_date), 6) as week_end_date,
  account_id,
  workspace_id,
  platform,
  sku,
  MAX(account_name) as account_name,
  MAX(account_executive) as account_executive,
  MAX(account_executive_manager) as account_executive_manager,
  MAX(workspace_name) as workspace_name,
  ROUND(SUM(revenue_dollars), 2) as total_revenue_dollars,
  ROUND(SUM(dbus_consumed), 2) as total_dbus_consumed,
  ROUND(AVG(list_price_per_dbu), 4) as avg_list_price_per_dbu,
  COUNT(DISTINCT consumption_date) as consumption_days_count,
  CURRENT_TIMESTAMP() as last_updated_timestamp
FROM ankit_yadav.demo.fact_consumption_daily
WHERE account_id LIKE 'ACCT%'
GROUP BY
  DATE_TRUNC('WEEK', consumption_date),
  account_id,
  workspace_id,
  platform,
  sku;

## 3. Verify Data

In [ ]:
%sql
-- Sample weekly data for Acme Corporation
SELECT * FROM ankit_yadav.demo.fact_consumption_weekly 
WHERE account_id = 'ACCT001'
ORDER BY week_start_date DESC, sku
LIMIT 20;

In [ ]:
%sql
-- Week-over-week revenue by account
SELECT 
  year_week,
  week_start_date,
  account_name,
  ROUND(SUM(total_revenue_dollars), 2) as weekly_revenue,
  ROUND(SUM(total_dbus_consumed), 2) as weekly_dbus
FROM ankit_yadav.demo.fact_consumption_weekly
WHERE account_id = 'ACCT001'
GROUP BY year_week, week_start_date, account_name
ORDER BY week_start_date DESC;

In [ ]:
%sql
-- Weekly totals by AEM
SELECT 
  year_week,
  account_executive_manager,
  COUNT(DISTINCT account_id) as account_count,
  ROUND(SUM(total_revenue_dollars), 2) as total_revenue,
  ROUND(SUM(total_dbus_consumed), 2) as total_dbus
FROM ankit_yadav.demo.fact_consumption_weekly
GROUP BY year_week, account_executive_manager
ORDER BY year_week DESC, total_revenue DESC;

In [ ]:
%sql
-- Record count summary
SELECT 
  COUNT(*) as total_records,
  COUNT(DISTINCT account_id) as unique_accounts,
  COUNT(DISTINCT year_week) as unique_weeks,
  MIN(week_start_date) as earliest_week,
  MAX(week_start_date) as latest_week,
  ROUND(SUM(total_revenue_dollars), 2) as total_revenue
FROM ankit_yadav.demo.fact_consumption_weekly;

In [ ]:
%sql
-- Week-over-week growth analysis
WITH weekly_totals AS (
  SELECT 
    year_week,
    week_start_date,
    SUM(total_revenue_dollars) as weekly_revenue
  FROM ankit_yadav.demo.fact_consumption_weekly
  GROUP BY year_week, week_start_date
)
SELECT 
  year_week,
  ROUND(weekly_revenue, 2) as weekly_revenue,
  ROUND(LAG(weekly_revenue) OVER (ORDER BY week_start_date), 2) as prev_week_revenue,
  ROUND(
    (weekly_revenue - LAG(weekly_revenue) OVER (ORDER BY week_start_date)) / 
    LAG(weekly_revenue) OVER (ORDER BY week_start_date) * 100, 2
  ) as wow_growth_pct
FROM weekly_totals
ORDER BY week_start_date DESC;